In [1]:
import datetime

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [2]:
# database connections
with open('../config_fill.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_aw = config['Adventure_Works']
    config_etl = config['ETL_PRO']

url_aw = f"mssql+pyodbc://@{config_aw['host']}/{config_aw['dbname']}?driver={config_aw['driver'].replace(' ', '+')}&trusted_connection={config_aw['trusted_connection']}"
url_etl = f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}"

aw_engine = create_engine(url_aw)
etl_engine = create_engine(url_etl)

In [3]:
query_territory = """
SELECT
    TerritoryID,
    Name AS TerritoryName,
    CountryRegionCode,
    [Group] AS TerritoryGroup
FROM Sales.SalesTerritory;
"""

df_territory = pd.read_sql(query_territory, aw_engine)
df_territory.head()


,TerritoryID,TerritoryName,CountryRegionCode,TerritoryGroup
0,1,Northwest,US,North America
1,2,Northeast,US,North America
2,3,Central,US,North America
3,4,Southwest,US,North America
4,5,Southeast,US,North America


In [4]:
df_territory = df_territory.fillna({
    "TerritoryName": "Unknown",
    "CountryRegionCode": "Unknown",
    "TerritoryGroup": "Unknown"
})


In [5]:
df_territory.rename(columns={
    "TerritoryID": "territory_id",
    "TerritoryName": "territory_name",
    "TerritoryGroup": "territory_group"
}, inplace=True)


In [6]:
df_territory.insert(0, "territory_key", range(1, len(df_territory) + 1))


In [7]:
df_territory.to_sql(
    "dim_territory",
    etl_engine,
    if_exists="replace",
    index=False
)


10